**Student side - Client**

In [22]:
import socket
import datetime
from dateutil import parser

In [25]:
host = '127.0.0.1'
clock_port = 3003
certificate_port = 5000

client_socket = socket.socket()
client_socket.connect((host, clock_port))

#Time at which client sent a request to clockServer
time_request_to_ClockServer = datetime.datetime.now().timestamp()   #T0
clock_data_recv = parser.parse(client_socket.recv(1024).decode())   #Tserver
#Time at which client received a response from the clockServer
time_response_received_from_ClockServer = datetime.datetime.now().timestamp() #T1
actual_time = datetime.datetime.now()
latency = time_response_received_from_ClockServer - time_request_to_ClockServer
client_time = clock_data_recv + datetime.timedelta(seconds= latency/2)
print('Synchronized time: ' + str(client_time))
print('Actual time: ' +  str(actual_time))
client_socket.close()

Synchronized time: 2023-04-24 09:59:17.158176
Actual time: 2023-04-24 09:59:17.158530
